In [3]:
#!pip install ../../../nlp-pie-taggers --upgrade
#!pip install https://github.com/PonteIneptique/pie/archive/improvement/AttentionDecoder.predict_max.zip --upgrade

!rm ./lemmatized/plain-text/*-pie*.txt

In [1]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=128, device="cuda", model_path=None)

2020-08-20 21:04:32,093 : Model /home/thibault/dev/these/these_env/lib/python3.6/site-packages/pie_extended/downloads/lasla/model.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 30f2c32b.


In [3]:
# Get the main object from the model (: data iterator + postprocesor
from pie_extended.models.lasla.imports import get_iterator_and_processor
import glob
import tqdm

for file in tqdm.tqdm(glob.glob("lemmatized/plain-text/*.txt")):
    try:
        if "-pie" not in file:
            iterator, processor = get_iterator_and_processor()
            tagger.tag_file(file, iterator=iterator, processor=processor)
    except Exception as E:
        print(file)
        print(E)

 48%|████▊     | 311/646 [42:22<45:38,  8.18s/it]  

lemmatized/plain-text/urn:cts:latinLit:stoa0275.stoa006.opp-lat1.txt
invalid literal for int() with base 10: '¹'


 68%|██████▊   | 437/646 [1:00:37<28:59,  8.32s/it]

KeyboardInterrupt: 

## Debug

In [2]:
# Get the main object from the model (: data iterator + postprocesor
from pie_extended.models.lasla.imports import get_iterator_and_processor

iterator, processor = get_iterator_and_processor()
file = "lemmatized/plain-text/urn:cts:latinLit:stoa0275.stoa006.opp-lat1.txt"
with open(file) as f:
    print(f.read())

tagger.tag_file(file, iterator=iterator, processor=processor)


[REF:chapter.1.1]     Si quid retro gestum est nobis adversus Marcionem, iam 
hinc viderit. Novam rem aggredimur ex vetere. Primum opusculum
quasi properatum pleniore postea compositione rescideram.
Hanc quoque nondum exemplariis suffectam fraude tunc fratris,
dehinc apostatae, amisi, qui forte descripserat quaedam mendosissime
et exhibuit frequentiae. Emendationis necessitas facta est.
Innovationis eius occasio aliquid adicere persuasit. Ita stilus iste
nunc de secundo tertius et de tertio iam hinc primus hunc opusculi
sui exitum necessario praefatur, ne quem varietas eius in disperso
reperta confundat. Pontus, qui dicitur Euxinus, natura negatur,
nomine illuditur. Ceterum hospitalem Pontum nec de situ aestimes;


ita ab humanioribus fretis nostris quasi quodam barbariae
suae pudore secessit. Gentes ferocissimae inhabitant; si tamen
habitatur in plaustro. Sedes incerta, vita cruda, libido promiscua
et plurimum nuda, etiam cum abscondunt, suspensis de iugo pharetris
indicibus, ne temer

'lemmatized/plain-text/urn:cts:latinLit:stoa0275.stoa006.opp-lat1-pie.txt'

In [ ]:
!mkdir -p lemmatized/tsv
!mv lemmatized/plain-text/*-pie.txt lemmatized/tsv/